In [1]:
import tensorflow as tf
import numpy as np
import sys
import os
import functools
import time
import config



#To change
from my_load_data import load_dataset_fn 
print("loaded libs")

loaded libs


In [2]:
print (len(os.listdir(config.PIC_SRC_DIR)))


params = {"result_dir": config.RESULT_DIR,
          "learning_rate": config.LR,
          "img_size": config.CNN_IN_HEIGHT,
          "num_channels":config.CNN_IN_CH,
          "num_classes": config.CNN_OUTPUT_SIZE,
          "batch_size": config.BATCH_SIZE,
          "total_steps": 1,
          "keep_probability": config.KEEP_PROB,
          "print_nth_step": config.PRINT_NTH}
print("loaded params")

1000
loaded params


In [15]:
#WRAPPERS FOR MODEL
def doublewrap(function):
    """
    A decorator of decorator, allowing use of lazy property if no arguments are provided
    """
    @functools.wraps(function)
    def decorator(*args, **kwargs):
        if len(args) == 1 and len(kwargs) == 0 and callable(args[0]):
            return function(args[0])
        else:
            return lambda wrapee: function(wrapee, *args, **kwargs)
    return decorator

@doublewrap
def define_scope(function, scope = None, *args, **kwargs):
    """
    Lazy decorator, optimizes code by loading class Model parts only once to memory
    Also its groups tf.Graph, in tensorboard into smaller, more readable parts
    """    
    attribute = '_cache_' + function.__name__
    name = scope or function.__name__
    
    @property
    @functools.wraps(function)
    def decorator(self):
        if not hasattr(self, attribute):
            #Sorting Graph by Var_scope
            with tf.variable_scope(name, *args, **kwargs):
                setattr(self, attribute, function(self))
                print("Initialized Model.{}".format(name))
        return getattr(self, attribute)
    
    return decorator


class Model(object):
    """
    Model of NN
    """
    #Variable of model
    #accuracy = tf.reduce
    
    
    
    def __init__(self, params):
        #self.permutation = np.random.permutation(range(len(os.listdir(config.PIC_SRC_DIR))))
        
        self.img_size = params["img_size"]
        self.num_channels = params["num_channels"]
        self.num_classes = params["num_classes"]
        self.lr = params["learning_rate"]
        self.keep_prob = params["keep_probability"]
        self.write_step = params["print_nth_step"]
        self.gstep = tf.Variable(0, dtype=tf.int32, 
                                trainable=False, name='global_step')
       
        
        
        self.training = False ### USE IN FUTURE
        
        #remove bellow
        self.data = tf.placeholder(dtype= tf.float32, shape=[None,256,256,3])
        self.target = tf.placeholder(dtype = tf.float32, shape=[None,5])
        self.permutation = np.random.permutation(range(1,6))
        self.data_iter = 1 
        self.total_epoch = 0
        
        #FUNCTIONS, don not remove
        self.load_data
        self.prediction
        self.loss_op
        self.optimize
   
        
        
    #FUNCTION DEFINITIONS
    def data_pipeline(self):
        """
        loading of TFRecords
        """
        pass
    
    
    #CHANGE LOAD DATA TO DATA_PIPELINE
    @define_scope    
    def load_data(self):
        """
        Serve random data each iteration
        """
        with tf.name_scope('Input'):
            self.data, self.target = (load_dataset_fn(self.permutation[self.data_iter]))
            print(type(self.data))
            print(self.data.shape)
            print(self.target.shape)
        
   
    @define_scope
    def prediction(self, if_training=True):
        """
        Main body of neural network, takes data and labels as input,
        returns feature map of photo
        """
        #INPUT LAYER
        input_layer = tf.reshape(self.data,[-1, self.img_size, self.img_size, self.num_channels]) 
        
        #1 conv layer
        conv1 = tf.layers.conv2d(inputs = self.data, 
                             filters = 16,
                             kernel_size = 5,
                             strides = 1,
                             padding = "same",
                             activation = tf.nn.relu)
        
        #1 pool layer, img size reduced by 1/4
        pool1 = tf.layers.max_pooling2d(inputs=conv1,
                                        pool_size = 2, 
                                        strides = 2,
                                        padding = "same")

        #2 conv layer
        conv2 = tf.layers.conv2d(inputs = pool1, 
                             filters = 32,
                             kernel_size = 5,
                             strides = 1,
                             padding = "same",
                             activation = tf.nn.relu)

        #2 pool overal image size reduced totaly by factor of 1/16
        pool2 = tf.layers.max_pooling2d(inputs = conv2,
                                        pool_size = 2, 
                                        strides = 2,
                                        padding = "same")
        
        pool2_flat = tf.reshape(pool2,[-1, 64*64*32])

        dense = tf.layers.dense(inputs = pool2_flat,
                            units = 128,
                            activation = tf.nn.relu)

        dense2 = tf.layers.dense(inputs = dense,
                             units = self.num_classes,
                             activation = tf.nn.relu)
       
        return tf.nn.softmax(dense2)
    
    
    @define_scope
    def loss_op(self):
        """
        loss
        """
        self.loss = tf.reduce_mean(
                    tf.nn.softmax_cross_entropy_with_logits_v2(logits = self.prediction,
                                                                labels = self.target))
        return self.loss
    
    
    @define_scope    
    def optimize(self):
        """
        Optimizer of model
        """
        optimizer = tf.train.AdamOptimizer(learning_rate=self.lr)
        
        return optimizer.minimize(self.loss)
        

        
    def train(self, sess, init, saver, writer,step, ckpt_dir):
        """
        Training op for model
        """
        start_time = time.time()
        sess.run(init)
        self.training = True
        total_loss = 0
        num_batches = 0
        try: 
            #train
            loss, _ = sess.run([self.loss_op, self.optimize])
            if (step + 1) % self.write_step == 0:
                print('Loss at step {0}: {1}'.format(step, loss))
            #save
           # writer.add_summary(summary, global_step=step)
            step += 1
            total_loss += loss
            num_batches += 1
        except KeyboardInterrupt:
            saver.save(sess, ckpt_dir, step)
            print("keyboard interrupt") 
        saver.save(sess, ckpt_dir, step)
        print('Average loss at epoch {0}: {1}'.format(step, total_loss/num_batches))
        print('Took: {0} seconds'.format(time.time() - start_time))
        return step
        

In [21]:
total_epoch = 10
def main():
    result_dir = config.RESULT_DIR
    
    #Check for dirs, if not present make them
    if not os.path.exists(result_dir):
        os.makedirs(result_dir)
    ckpt_dir=os.path.join(result_dir,"ckpt")
    if not os.path.exists(ckpt_dir):
        os.makedirs(ckpt_dir)
    
    graph = tf.Graph()
    #Get name as default graph
    with graph.as_default():

        print("Starting Session")
        #Assign name to session, assign it's default graph as graph
        with tf.Session(graph=graph) as sess:
            
            #Creating summary writer 
            writer = tf.summary.FileWriter(ckpt_dir, graph=graph)
                
            #Initialization of Model, load all Model functions returning variables
            model = Model(params)
            
            #Assign Initializer
            init = tf.global_variables_initializer()
            
            #Creating save for model session for future saving and restoring model
            saver = tf.train.Saver()
            
            #Loading last checkpoint
            ckpt = tf.train.get_checkpoint_state(result_dir)
            if ckpt and ckpt.model_checkpoint_path:
                #if ckpt found load it and load global step
                saver.restore(sess, ckpt.model_checkpoint_path)
                print("Found checkpoint")
                step = model.gstep
            else: step = 1
            
                    
            #Training
            print("Starting Training")
           
            for epoch in range(total_epoch):
                step = model.train(sess, init, saver, writer, step, ckpt_dir)
              
            
        
        
        print("Finnished session")
        #Merge all summaries
        #writer.flush()
        writer.add_graph(graph)
        writer.close()
        print("Closed summary, work finnished")

In [22]:
tf.reset_default_graph()
if __name__ == '__main__':
     main()

Starting Session
<class 'tensorflow.python.framework.ops.Tensor'>
(20, 256, 256, 3)
(20, 5)
Initialized Model.load_data
Initialized Model.prediction
Initialized Model.loss_op
Initialized Model.optimize
INFO:tensorflow:Restoring parameters from results\ckpt-2
Found checkpoint
Starting Training
Average loss at epoch Tensor("add_1:0", shape=(), dtype=int32): 18.353199005126953
Took: 7.921292543411255 seconds
Average loss at epoch Tensor("add_3:0", shape=(), dtype=int32): 20.5350341796875
Took: 5.852907419204712 seconds
Average loss at epoch Tensor("add_5:0", shape=(), dtype=int32): 16.682226181030273
Took: 5.924044847488403 seconds
Average loss at epoch Tensor("add_7:0", shape=(), dtype=int32): 19.208091735839844
Took: 5.521616458892822 seconds
Average loss at epoch Tensor("add_9:0", shape=(), dtype=int32): 21.051912307739258
Took: 6.231894254684448 seconds
Average loss at epoch Tensor("add_11:0", shape=(), dtype=int32): 15.79419231414795
Took: 6.030658960342407 seconds
Average loss at ep

SystemError: <built-in function WritableFile_Close> returned a result with an error set